# Library import 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import logging
import numpy as np
import re
import tqdm
from threading import *
import queue
import time

# Target links

In [2]:
start=336277
end = 339000
link_list=[]
for i in range(start,end):
    link_list.append('http://www.newsmth.net/nForum/article/Intern/'+str(i)+'?ajax' )

# Scrapy Funtion

## Process extends Thread Class

In [3]:
class ShuimuSpider(Thread):
    def __init__(self,url,q):
        super(ShuimuSpider,self).__init__()
        self.url=url
        self.q=q
        
    def run(self):
        self.parse_page()
        
    def send_request(self,url):
        try:
            html = requests.get(url,timeout=2)
        except Exception as e:
            print("fail at %s" % url)
        else:
            return html
    def parse_page(self):
        response = self.send_request(self.url)
        if response.status_code==200:
            response_bs=response.content
            try:
                bs = BeautifulSoup(response_bs,'html.parser')


                text=''
                store_data={
                    'time':[],
                    'title':[],
                    'detail':[]
                }
                text = bs.p.text
                # .p 对应html中的<p>, 观察帖子不难发现，几乎所有的招聘正文都在帖子的<p>内
                # .text 对应输出文本

                #分别通过特殊字符串的位置给标题、时间、正文定位
                #position for title:
                Title_Position1=text.find('标  题: ')+len('标  题: ')
                Title_Position2=text.find('  发信站: 水木社区')
                Title = text[Title_Position1:Title_Position2]
                #text[x:y]进行切片

                #position for time:
                Time_Position1=text.find('  发信站: 水木社区')+len('  发信站: 水木社区')
                Time_Position2=text.find('), 站内')+1
                Time=text[Time_Position1:Time_Position2]

                #position for detail
                Detail_Position1=text.find('), 站内')+len('), 站内')+4
                Detail_Position2=text.find('※ 来源:·水木社区')
                Detail=text[Detail_Position1:Detail_Position2]

                store_data['time'].append(Time)
                store_data['title'].append(Title)
                store_data['detail'].append(Detail)
                #字典赋值
                #运行成功，index序列加一

                self.q.put(store_data)
            except:
                return
        else:
            return

## Scrapy function with multi-threading

In [4]:
def Fast_Scrapy(x,y,store_data):
    start = time.time()
    q= queue.Queue(0)

    Thread_list=[]
    for url in link_list[x:y]:
        p = ShuimuSpider(url,q)
        p.start()
        Thread_list.append(p)

    for i in Thread_list:
        i.join()

    while not q.empty():
        data = q.get()

        store_data['time'].append(data['time'])
        store_data['title'].append(data['title'])
        store_data['detail'].append(data['detail'])
    end = time.time()
    print('当前Process + Queue多线程爬虫的总时间为：', end-start)

## Scrapy RUN

In [5]:
def Scrapy():
    store_data={
        'time':[],
        'title':[],
        'detail':[]
    }
    if end-start>150:
        period = np.arange(0,end-start,150)

        for i in tqdm.tqdm(period):
            Fast_Scrapy(i,i+150,store_data)
        Fast_Scrapy(i,end,store_data)
    else:
        Fast_Scrapy(start,end,store_data)
        
    df = pd.DataFrame(store_data)
    df.to_csv('../datasets/'+str(start)+'-'+str(end)+'.csv')
    return df

In [6]:
%%time
df = Scrapy()

  5%|████▎                                                                              | 1/19 [00:03<01:05,  3.65s/it]

当前Process + Queue多线程爬虫的总时间为： 3.642998456954956


 11%|████████▋                                                                          | 2/19 [00:06<00:58,  3.44s/it]

当前Process + Queue多线程爬虫的总时间为： 3.28900408744812


 16%|█████████████                                                                      | 3/19 [00:10<00:53,  3.37s/it]

...

 95%|█████████████████████████████████████████████████████████████████████████████▋    | 18/19 [00:58<00:03,  3.25s/it]

当前Process + Queue多线程爬虫的总时间为： 3.1789627075195312


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:59<00:00,  3.13s/it]

当前Process + Queue多线程爬虫的总时间为： 0.629037618637085


当前Process + Queue多线程爬虫的总时间为： 0.6779940128326416
Wall time: 1min


# Data Cleaning

## Remove '[]'

In [7]:
# remove '[]'
for col in ['time','title','detail']:
    df[col]=df[col].apply(lambda x:x[0])

## Transfer to pd.datetime

In [8]:
# transfer to pd.datetime
df['time']=(df['time'].str[2:12]+df['time'].str[21:-1])
df['time']=pd.to_datetime(df['time'],errors='coerce')
df['time']=df['time'].fillna(method='ffill')

## remove data with extremely short title

In [10]:
df = df[df['title'].str.len()>5]

## 去重

In [11]:
df = df[~df['title'].duplicated()]
df.reset_index(drop=True,inplace=True)

## Remove datas without a delivering email

In [12]:
df_copy =df.copy()

In [21]:
df_copy=df_copy[~df_copy['title'].str.contains('个人提供')]
df_copy=df_copy[~df_copy['title'].str.contains('博后')]

In [13]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern,flags=re.IGNORECASE)
df['Email']='NaN'
for i in range(len(df)):
    try:
        df.iloc[i,3]=regex.findall(df['detail'][i])
    except:
        continue
df.replace(to_replace='NaN',value = np.nan,regex=True,inplace =True)
df.dropna(axis=0,how='any',inplace =True)
df.reset_index(drop=True,inplace=True)

## output as text file

In [30]:
def output(df):
    text=''
    for i in range(len(df)):
        text += str(df.index[i])+' '
        text += df['title'][i]+'\n'
        text += df['detail'][i]+'\n'
        #text += df['Email'][i]+'\n\n\n'
    
    with open('2021-05-06.txt','w',encoding='utf-8') as f:
        f.write(text)

In [31]:
output(df_copy)

# Classify

## Load Classify Model

In [13]:
import joblib
clf_model=joblib.load('../models/Classification_model_5_n_estimator=100_pca=100.joblib')

## Predict function

In [24]:
mark = ['人力资源', '其他', '券商投行基金', '前端&测试', '投资咨询实习生', '数据分析挖掘', '研发开发',
       '算法实习', '运营实习', '量化交易']

In [25]:
import jieba_fast

stopwords = pd.read_csv('../stopwords/baidu_stopwords.txt',
                        quoting=3,sep='\t',names=['stopword'],encoding='utf-8')
stopwords = stopwords['stopword'].values

def pre_input(input):
    sentences=[]
    segs=jieba_fast.cut(input,cut_all=False)
    segs = list(filter(lambda x:x.strip(),segs))
    segs=list(filter(lambda x:((x not in stopwords) and x != 'xa0' ),segs))
    sentences.append(str([x for x in segs]))
    return sentences

def predict_df(x):
    return mark[clf_model.predict(pre_input(x))[0]]

## predict for dataframe

In [26]:
import re
req = re.compile('(职责|内容|要求)(.|\n)*')
df['requirement']=np.zeros(len(df))
def find_req(x):
    try: 
        re.search(req,x).span()
        return x[re.search(req,x).span()[0] : re.search(req,x).span()[1]]
    except:
        return ''
df['requirement']=df['detail'].apply(find_req)
df = df[df['requirement']!='']

In [27]:
df['category']=np.zeros(len(df))

In [28]:
df.reset_index(drop=True,inplace=True)

In [29]:
%%time
df['category']=(df['title']+df['requirement']).apply(lambda x:predict_df(x))

Wall time: 50.7 s


In [1]:
df[50:60][['title','category']]

NameError: name 'df' is not defined

数据集7000   岗位种类少 0.803